In [ ]:
from __future__ import annotations

import duckdb
import pandas as pd

ROOT = "."  # run from btc_data/

PATHS = {
    "spot_daily":        f"{ROOT}/coingecko/market_daily/*.parquet",
    "spot_hourly":       f"{ROOT}/coingecko/market_hourly_chunks/vs=usd/*.parquet",
    "ohlc_daily":        f"{ROOT}/coingecko/ohlc_daily_chunks/vs=usd/*.parquet",
    "exch_vol":          f"{ROOT}/coingecko/exchange_volume_chunks/exchange=*/exchange_volume_*.parquet",
    "global_snap":       f"{ROOT}/coingecko/global_snapshots/*.parquet",
    "deriv_snap":        f"{ROOT}/coingecko/derivatives_btc_snapshots/*.parquet",
    "cex_trades":        f"{ROOT}/cex/cex_trades/*.parquet",
    "perp_funding_oi":   f"{ROOT}/perp/btc_funding_oi/*.parquet",
}

con = duckdb.connect(database=":memory:")
con.execute("PRAGMA threads=4;")

def profile(name: str, glob_path: str) -> pd.DataFrame:
    # Show schema + basic coverage, without materializing full tables in Python.
    q = f"""
    WITH t AS (SELECT * FROM read_parquet('{glob_path}'))
    SELECT
        '{name}' AS dataset,
        COUNT(*) AS rows,
        MIN(time) AS min_time,
        MAX(time) AS max_time
    FROM t
    """
    # Some tables may not have "time" (e.g. derivatives uses snapshot_time or similar).
    # We'll try "time" first; on failure, fall back to "snapshot_time" then "timestamp".
    try:
        return con.execute(q).df()
    except duckdb.Error:
        for alt in ["snapshot_time", "timestamp"]:
            q2 = f"""
            WITH t AS (SELECT * FROM read_parquet('{glob_path}'))
            SELECT
                '{name}' AS dataset,
                COUNT(*) AS rows,
                MIN({alt}) AS min_time,
                MAX({alt}) AS max_time
            FROM t
            """
            try:
                return con.execute(q2).df()
            except duckdb.Error:
                continue
        # No time-like column found; just row count.
        q3 = f"""
        WITH t AS (SELECT * FROM read_parquet('{glob_path}'))
        SELECT '{name}' AS dataset, COUNT(*) AS rows
        FROM t
        """
        return con.execute(q3).df()

def show_columns(name: str, glob_path: str) -> pd.DataFrame:
    q = f"""
    SELECT column_name, data_type
    FROM duckdb_columns()
    WHERE table_name = 'read_parquet'
    """
    # Trick: DESCRIBE read_parquet(...) is easiest
    return con.execute(f"DESCRIBE SELECT * FROM read_parquet('{glob_path}')").df()

def dedupe_time_key(glob_path: str, key_cols: list[str]) -> str:
    """
    Returns a DuckDB subquery that keeps 1 row per key. Useful for chunk overlaps.
    """
    key = ", ".join(key_cols)
    return f"""
    (
      SELECT * EXCLUDE(rn)
      FROM (
        SELECT *, row_number() OVER (PARTITION BY {key} ORDER BY {key}) AS rn
        FROM read_parquet('{glob_path}')
      )
      WHERE rn = 1
    )
    """

# -----------------------------
# 1) PROFILE ALL DATASETS
# -----------------------------
profiles = []
for k, p in PATHS.items():
    profiles.append(profile(k, p))

print("\n=== DATASET PROFILE ===")
print(pd.concat(profiles, ignore_index=True).to_string(index=False))

# Optional: show schemas for the core ones
for k in ["spot_daily", "spot_hourly", "exch_vol", "perp_funding_oi", "deriv_snap"]:
    print(f"\n=== COLUMNS: {k} ===")
    try:
        print(show_columns(k, PATHS[k]).to_string(index=False))
    except Exception as e:
        print(f"(could not describe {k}: {e})")

# -----------------------------
# 2) QA CHECKS: DUPES + GAPS
# -----------------------------
# Spot hourly: check missing hours and duplicates after dedupe
spot_hourly = dedupe_time_key(PATHS["spot_hourly"], ["time"])  # time should be unique per hour
q_gaps = f"""
WITH t AS (
  SELECT time::TIMESTAMP AS time, price
  FROM {spot_hourly}
),
d AS (
  SELECT
    time,
    LAG(time) OVER (ORDER BY time) AS prev_time
  FROM t
)
SELECT
  COUNT(*) FILTER (WHERE prev_time IS NULL) AS first_row,
  COUNT(*) FILTER (WHERE prev_time IS NOT NULL AND (time - prev_time) != INTERVAL 1 HOUR) AS non_1h_steps,
  MIN(time - prev_time) FILTER (WHERE prev_time IS NOT NULL) AS min_step,
  MAX(time - prev_time) FILTER (WHERE prev_time IS NOT NULL) AS max_step
FROM d
"""
print("\n=== QA: SPOT HOURLY GAPS ===")
print(con.execute(q_gaps).df().to_string(index=False))

# Exchange volume: chunks overlap at boundaries; dedupe by (exchange,time)
exch_vol = dedupe_time_key(PATHS["exch_vol"], ["exchange", "time"])
q_exch = f"""
WITH t AS (
  SELECT exchange, time::TIMESTAMP AS time, volume_btc
  FROM {exch_vol}
)
SELECT exchange, COUNT(*) AS rows, MIN(time) AS min_time, MAX(time) AS max_time
FROM t
GROUP BY 1
ORDER BY 1
"""
print("\n=== EXCHANGE VOLUME COVERAGE (DEDUPED) ===")
print(con.execute(q_exch).df().to_string(index=False))

# -----------------------------
# 3) BUILD ANALYTICAL VIEWS
# -----------------------------
# (A) Daily spot returns
spot_daily = dedupe_time_key(PATHS["spot_daily"], ["time"])
q_spot_daily = f"""
WITH t AS (
  SELECT time::DATE AS day, price
  FROM {spot_daily}
),
r AS (
  SELECT
    day,
    price,
    LN(price / LAG(price) OVER (ORDER BY day)) AS log_ret_1d
  FROM t
)
SELECT * FROM r ORDER BY day
"""
spot_daily_df = con.execute(q_spot_daily).df()

# (B) Exchange volume converted to USD using daily spot price
#     Align exchange volume (likely daily points) to day and multiply by spot price.
q_exch_usd = f"""
WITH px AS (
  SELECT time::DATE AS day, price
  FROM {spot_daily}
),
v AS (
  SELECT exchange, time::DATE AS day, volume_btc
  FROM {exch_vol}
),
j AS (
  SELECT v.exchange, v.day, v.volume_btc, px.price, (v.volume_btc * px.price) AS volume_usd
  FROM v
  JOIN px USING (day)
)
SELECT * FROM j
"""
exch_usd_df = con.execute(q_exch_usd).df()

# (C) Funding vs subsequent returns (simple example)
#     Your perp_funding_oi parquet may have mixed granularities; we attempt to normalize to hourly.
perp = dedupe_time_key(PATHS["perp_funding_oi"], ["exchange", "symbol", "time"])
q_funding = f"""
WITH t AS (
  SELECT
    time::TIMESTAMP AS time,
    exchange,
    symbol,
    funding_rate
  FROM {perp}
  WHERE funding_rate IS NOT NULL
),
px AS (
  SELECT
    time::TIMESTAMP AS time,
    price,
    LN(price / LAG(price) OVER (ORDER BY time)) AS log_ret_1h
  FROM {spot_hourly}
),
j AS (
  SELECT
    t.time,
    t.exchange,
    t.symbol,
    t.funding_rate,
    px.log_ret_1h,
    LEAD(px.log_ret_1h, 1) OVER (ORDER BY t.time) AS fwd_log_ret_1h
  FROM t
  JOIN px USING (time)
)
SELECT * FROM j
"""
try:
    funding_join_df = con.execute(q_funding).df()
except duckdb.Error as e:
    funding_join_df = None
    print("\nNOTE: funding join failed (schema mismatch is possible). Error:\n", e)

# -----------------------------
# 4) SIMPLE OUTPUT + PLOTS
# -----------------------------
print("\n=== DAILY SPOT SAMPLE ===")
print(spot_daily_df.head(5).to_string(index=False))

print("\n=== EXCHANGE USD VOLUME SAMPLE ===")
print(exch_usd_df.head(5).to_string(index=False))

if funding_join_df is not None:
    print("\n=== FUNDING JOIN SAMPLE ===")
    print(funding_join_df.head(5).to_string(index=False))

    # quick correlation (overall)
    corr = funding_join_df[["funding_rate", "fwd_log_ret_1h"]].corr(numeric_only=True).iloc[0,1]
    print(f"\nCorr(funding_rate, next_hour_return) = {corr:.6f}")

# Optional plot
import matplotlib.pyplot as plt

# Plot BTC daily price + daily log returns (two separate figures, no style/colors forced)
fig = plt.figure()
plt.plot(spot_daily_df["day"], spot_daily_df["price"])
plt.title("BTC spot price (daily)")
plt.xlabel("day")
plt.ylabel("price")
plt.tight_layout()

fig = plt.figure()
plt.plot(spot_daily_df["day"], spot_daily_df["log_ret_1d"])
plt.title("BTC log returns (1d)")
plt.xlabel("day")
plt.ylabel("log_ret_1d")
plt.tight_layout()

plt.show()


: 